In [4]:
import cv2, os
img_last = None # 이전 프레임을 저장할 변수
no = 0 # 이미지 장 수
save_dir = "./exfish" # 저장 디렉터리 이름
cap = cv2.VideoCapture("fish.mp4") # 동영상 파일로부터 입력받기 --- (*1)
while True:
    is_ok, frame = cap.read() # 이미지 추출하기
    if not is_ok: break
    frame = cv2.resize(frame, (640, 360))    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백 이미지로 변환하기 --- (*2)
    gray = cv2.GaussianBlur(gray, (15, 15), 0)
    rec, img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)    
    if not img_last is None: # 차이 확인하기
        frame_diff = cv2.absdiff(img_last, img_b) # --- (*3)
        cnts, hierachy = cv2.findContours(frame_diff,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)        
        for pt in cnts: # 차이가 있는 부분을 파일로 출력하기 --- (*4)
            x, y, w, h = cv2.boundingRect(pt)
            if w < 100 or w > 500: continue # 노이즈 제거하기           
            imgex = frame[y:y+h, x:x+w] # 추출한 영역 저장하기
            outfile = save_dir + "/" + str(no) + ".jpg"
            cv2.imwrite(outfile, imgex)
            no += 1
    img_last = img_b
cap.release()
print("ok")

ok


In [6]:
import glob
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

In [8]:
image_size = (64,32)
path = 'D:/jupyter/ai'
path_fish = path + '/fish'
path_nofish = path + '/nofish'

x = []
y = []

def read_dir (path, label):
    files = glob.glob(path + "/*.jpg")
    for f in files:
        img = cv2.imread(f)
        img = cv2.resize(img, image_size)
        img_data = img.reshape(-1, )
        x.append(img_data)
        y.append(label)
read_dir(path_nofish, 0)
read_dir(path_fish, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))
joblib.dump(clf, 'fish.pkl')

D:\Program Files\anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9671361502347418


['fish.pkl']

In [9]:
import cv2, os, copy
from sklearn.externals import joblib
clf = joblib.load('fish.pkl')
output_dir = './bestshot'
img_last = None
fish_th = 3

In [14]:
count = 0
frame_count = 0
if not os.path.isdir(output_dir): os.mkdir(output_dir)
cap = cv2.VideoCapture("fish.mp4")
while True:
    is_ok, frame = cap.read()
    if not is_ok : break
    frame = cv2.resize(frame, (640,360))
    frame2 = copy.copy(frame)
    frame_count += 1
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (15, 15), 0)
    rec, img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    if not img_last is None:
        frame_diff = cv2.absdiff(img_last, img_b)
        cnts, hierachy = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        fish_count = 0
        for pt in cnts:
            x, y, w, h = cv2.boundingRect(pt)
            if w < 100 or w > 500: continue
            imgex = frame[y:y+h, x:x+w]
            imagex = cv2.resize(imgex, (64, 32))
            image_data = imagex.reshape(-1, )
            pred_y = clf.predict([image_data])
            if pred_y[0] == 1:
                fish_count += 1
                cv2.rectangle(frame2, (x,y), (x+w, y+h), (0,255,0), 2)
        if fish_count > fish_th:
            fname = output_dir + "/fish" + str(count) + ".jpg"
            cv2.imwrite(fname, frame)
            count+= 1
    cv2.imshow('FISH!', frame2)
    if cv2.waitKey(1) == 13: break
    img_last = img_b
cap.release()
cv2.destroyAllWindows()
print("ok", count, "/", frame_count)

ok 13 / 1983
